---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler

import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [11]:
def graph_identification():
    
    # Your Code Here
    
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

In [12]:
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
def salary_predictions():
    
## Feature Selection

    data = pd.DataFrame(index=G.nodes())
    data['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    data['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    data['Clustering'] = pd.Series(nx.clustering(G))
    data['Degree'] = pd.Series(G.degree())
    data['Degree_centrality'] = pd.Series(nx.degree_centrality(G))
    data['Closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    data['Betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    data['PageRank'] = pd.Series(nx.pagerank(G))

## Train and Test set

    No_null_data = data[data['ManagementSalary'].notnull()] 
    y_train = No_null_data['ManagementSalary']
    X_train = No_null_data.drop('ManagementSalary',axis=1)

    Null_data = data[data['ManagementSalary'].isnull()]  
    y_test = Null_data['ManagementSalary']
    X_test = Null_data.drop('ManagementSalary',axis=1)
    
    scaler = MinMaxScaler()
    X_train_sca = scaler.fit_transform(X_train)
    X_test_sca = scaler.transform(X_test)

## Training

    Method = RandomForestClassifier(n_estimators=51, max_depth=5).fit(X_train_sca, y_train)
    Predicted = Method.predict_proba(X_test_sca)[:, 1]
    
    return  pd.Series(Predicted,X_test.index)

In [6]:
salary_predictions()

1       0.066009
2       0.952734
5       1.000000
8       0.236342
14      0.109327
18      0.125218
27      0.144824
30      0.480275
31      0.362182
34      0.137443
37      0.145064
40      0.120238
45      0.075137
54      0.409139
55      0.396285
60      0.260785
62      1.000000
65      0.919400
77      0.028712
79      0.153618
97      0.021957
101     0.020001
103     0.456750
108     0.111390
113     0.216471
122     0.008805
141     0.296012
142     1.000000
144     0.031464
145     0.362688
          ...   
913     0.012858
914     0.014755
915     0.006962
918     0.092473
923     0.021288
926     0.045520
931     0.011382
934     0.007919
939     0.005114
944     0.008136
945     0.021288
947     0.046246
950     0.130557
951     0.008533
953     0.009720
959     0.007443
962     0.005114
963     0.249248
968     0.043911
969     0.051153
974     0.022517
984     0.008534
987     0.023224
989     0.039813
991     0.067598
992     0.008912
994     0.008136
996     0.0069

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [7]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})

In [8]:
##preds = list(nx.jaccard_coefficient(G, [(node_1, node)]))
##preds[0][2]

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [9]:
def new_connections_predictions():
    
    ## Add Atribute

    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
        
## Feature Selection

    future_connections['Jaccard Coefficient'] = [i[2] for i in nx.jaccard_coefficient(G, future_connections.index)]
    future_connections['Resource Allocation'] = [i[2] for i in nx.resource_allocation_index(G, future_connections.index)]
    future_connections['Adamic Adar'] = [i[2] for i in nx.adamic_adar_index(G, future_connections.index)]
    future_connections['Preferential Attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    future_connections['Ra_soundarajan_hopcroft'] = [i[2] for i in nx.ra_index_soundarajan_hopcroft(G, future_connections.index)]

## Train and Test set

    No_null_data = future_connections[future_connections['Future Connection'].notnull()] 
    y_train = No_null_data['Future Connection']
    X_train = No_null_data.drop('Future Connection',axis=1)

    Null_data = future_connections[future_connections['Future Connection'].isnull()]  
    y_test = Null_data['Future Connection']
    X_test = Null_data.drop('Future Connection',axis=1)
    
    scaler = MinMaxScaler()
    X_train_sca = scaler.fit_transform(X_train)
    X_test_sca = scaler.transform(X_test)

## Training

    Method = RandomForestClassifier(n_estimators=51, max_depth=5).fit(X_train_sca, y_train)
    Predicted = Method.predict_proba(X_test_sca)[:, 1]
    
    return  pd.Series(Predicted,X_test.index)

In [10]:
new_connections_predictions()

(107, 348)    0.027987
(542, 751)    0.014289
(20, 426)     0.576253
(50, 989)     0.014289
(942, 986)    0.014289
(324, 857)    0.014289
(13, 710)     0.157047
(19, 271)     0.145798
(319, 878)    0.014289
(659, 707)    0.014289
(49, 843)     0.014289
(208, 893)    0.014289
(377, 469)    0.016096
(405, 999)    0.017773
(129, 740)    0.014426
(292, 618)    0.038207
(239, 689)    0.014289
(359, 373)    0.015338
(53, 523)     0.136374
(276, 984)    0.014289
(202, 997)    0.014289
(604, 619)    0.068180
(270, 911)    0.014289
(261, 481)    0.054831
(200, 450)    0.969767
(213, 634)    0.014289
(644, 735)    0.089718
(346, 553)    0.014289
(521, 738)    0.014289
(422, 953)    0.017089
                ...   
(672, 848)    0.014289
(28, 127)     0.949097
(202, 661)    0.014289
(54, 195)     0.978846
(295, 864)    0.014289
(814, 936)    0.014289
(839, 874)    0.014289
(139, 843)    0.014289
(461, 544)    0.014289
(68, 487)     0.014289
(622, 932)    0.014289
(504, 936)    0.015626
(479, 528) 